In [33]:
import numpy as np
import pandas as pd

from queue import PriorityQueue
import networkx as nx
from pyvis.network import Network

In [2]:
movies_df = pd.read_csv('data/movies_data.csv')
cast_df = pd.read_csv('data/cast_data.csv')
cast_and_movies_df = pd.read_csv('data/cast_and_movies_data.csv')

In [34]:
movies_df.head()

,id,title,year
0,15724,Dama de noche,1993
1,23331,Pesn o geroyakh,1983
2,31458,El huésped del sevillano,1970
3,35423,Kate & Leopold,2001
4,36606,"Another Time, Another Place",1983


In [4]:
cast_df.head()

,id,name,birth
0,1,Fred Astaire,1899.0
1,2,Lauren Bacall,1924.0
2,3,Brigitte Bardot,1934.0
3,4,John Belushi,1949.0
4,5,Ingmar Bergman,1918.0


In [5]:
cast_and_movies_df.head()

,person_id,movie_id
0,844752,15724
1,869732,15724
2,194720,15724
3,650495,15724
4,8738,31458


# Graph construction

In [6]:
import queue

### I got the code of the cell below from GeeksForGeeks:

https://www.geeksforgeeks.org/minimum-number-of-edges-between-two-vertices-of-a-graph/

In [7]:
def minEdgeBFS(edges, u, v, n):
    
    # visited[n] for keeping track
    # of visited node in BFS
    visited = [0] * n

    # Initialize distances as 0
    distance = [0] * n

    # queue to do BFS.
    Q = queue.Queue()
    distance[u] = 0

    Q.put(u)
    visited[u] = True
    while (not Q.empty()):
        x = Q.get()
        
        for i in range(len(edges[x])):
            if (visited[edges[x][i]]):
                continue

            # update distance for i
            distance[edges[x][i]] = distance[x] + 1
            Q.put(edges[x][i])
            visited[edges[x][i]] = 1
    return distance[v]

# function for addition of edge
def addEdge(edges, u, v):
    edges[u].append(v)
    edges[v].append(u)

In [8]:
kevin_bacon_id = 102

### Here, I am sampling only 5% of the data, for demonstration purposes.

My computer couldn't build the graph with all the actors, it's just too much for it!

In [41]:
kevin_bacon_df = cast_and_movies_df[cast_and_movies_df['person_id'] == kevin_bacon_id]
cast_and_movies_df_without_bacon = cast_and_movies_df[cast_and_movies_df['person_id'] != kevin_bacon_id]

sample_ids = pd.Series(cast_and_movies_df_without_bacon['person_id'].unique()).sample(frac=0.05).unique()

sampled_cast_movies_df = cast_and_movies_df[cast_and_movies_df['person_id'].isin(sample_ids)]
sampled_cast_movies_df = pd.concat([sampled_cast_movies_df, kevin_bacon_df])

In [42]:
person_nodes = {}
for i, value in enumerate(sampled_cast_movies_df['person_id'].sort_values().unique()):
    person_nodes[i] = value

In [43]:
len(person_nodes)

27036

In [44]:
movie_edges = {}
for i, value in enumerate(sampled_cast_movies_df['movie_id'].sort_values().unique()):
    movie_edges[i] = value

In [45]:
len(movie_edges)

53493

### The cells below are meant to be run only once, to create the graph, and it takes a while to finish.

In [46]:
# n = len(movie_edges)
# edges = [[] for i in range(n)]

# for i in sampled_cast_movies_df.values:
#     p_node_value = i[0]
#     m_edge_value = i[1]

#     p_node = list(person_nodes.keys())[list(person_nodes.values()).index(p_node_value)]
#     m_edge = list(movie_edges.keys())[list(movie_edges.values()).index(m_edge_value)]
    
#     addEdge(edges, m_edge, p_node)

In [49]:
# pd.DataFrame(edges).to_csv('data/graph.csv')

In [50]:
# sampled_cast_movies_df.to_csv('data/sampled_cast_and_movies_data.csv', index=False)

# Executing Breadth-First Search (BFS)

In [51]:
sampled_cast_movies_df = pd.read_csv('data/sampled_cast_and_movies_data.csv')

In [52]:
graph = pd.read_csv('data/graph.csv', sep=',').drop('Unnamed: 0', axis=1)
graph = [graph.iloc[i].dropna().astype(int).to_list() for i in graph.index]

In [130]:
cast_df[cast_df['id'].isin(sampled_cast_movies_df['person_id'].unique())][
    'name'
].sort_values().values

array(["'Crazy' Bob Cook", "'Dirty Dave' Grimsley", "'University' Jeevan",
       ..., 'Ülle Lichtfeldt', 'Ülo Voitka', 'Þórey Birgisdóttir'],
      dtype=object)

In [120]:
example_id = sampled_cast_movies_df.sample(1).values[0][0]
example_name = cast_df[cast_df['id'] == example_id]['name'].iloc[0]
example_id_graph_node = list(person_nodes.keys())[list(person_nodes.values()).index(example_id)]

In [121]:
kevin_bacon_id_graph_node = list(person_nodes.keys())[list(person_nodes.values()).index(kevin_bacon_id)]

In [122]:
# randomly chosen actor
cast_df[cast_df['id'] == example_id]

,id,name,birth
424,437,Woody Harrelson,1961.0


In [124]:
n = len(graph)
result = minEdgeBFS(graph, example_id_graph_node, kevin_bacon_id_graph_node, n)
print(f"{example_name} is {result} movies away from Kevin Bacon.")

Woody Harrelson is 4 movies away from Kevin Bacon.
